In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv1D, MaxPooling1D
import pandas as pd
import numpy as np
from scipy import stats
import os

In [2]:
har_df = pd.read_csv("./task1.csv")

In [3]:
df = har_df

In [9]:
class CustomLabelEncoder:
    def __init__(self):
        self.class_mapping = {
            'sitting_standing': 0,
            'lyingLeft': 1,
            'lyingRight': 2,
            'lyingBack': 3,
            'lyingStomach': 4,
            'normalWalking': 5,
            'running': 6,
            'descending': 7,
            'ascending': 8,
            'shuffleWalking': 9,
            'miscMovement':10
        }

    def fit_transform(self, y):
        return [self.class_mapping[cls] for cls in y]

    def inverse_transform(self, y):
        reverse_mapping = {v: k for k, v in self.class_mapping.items()}
        return [reverse_mapping[val] for val in y]

In [10]:
le = CustomLabelEncoder()
df['label'] = le.fit_transform(df.activity)

In [15]:
window_size = 125

stride = 10  # Adjust this as needed

# Create sliding windows and labels
X, y = [], []

for i in range(0, len(df) - window_size, stride):
    window_data = df.iloc[i:i + window_size]
    X.append(window_data.drop(['activity', 'timestamp'], axis=1).values)
    y.append(stats.mode(window_data['label']).mode)
    # y.append(window_data['Activity'].values[-1])


In [16]:
y

[array([8], dtype=int64),
 array([8], dtype=int64),
 array([8], dtype=int64),
 array([8], dtype=int64),
 array([8], dtype=int64),
 array([8], dtype=int64),
 array([8], dtype=int64),
 array([8], dtype=int64),
 array([8], dtype=int64),
 array([8], dtype=int64),
 array([8], dtype=int64),
 array([8], dtype=int64),
 array([8], dtype=int64),
 array([8], dtype=int64),
 array([8], dtype=int64),
 array([8], dtype=int64),
 array([8], dtype=int64),
 array([8], dtype=int64),
 array([8], dtype=int64),
 array([8], dtype=int64),
 array([8], dtype=int64),
 array([8], dtype=int64),
 array([8], dtype=int64),
 array([8], dtype=int64),
 array([8], dtype=int64),
 array([8], dtype=int64),
 array([8], dtype=int64),
 array([8], dtype=int64),
 array([8], dtype=int64),
 array([8], dtype=int64),
 array([8], dtype=int64),
 array([8], dtype=int64),
 array([8], dtype=int64),
 array([8], dtype=int64),
 array([8], dtype=int64),
 array([8], dtype=int64),
 array([8], dtype=int64),
 array([8], dtype=int64),
 array([8], 

In [17]:
X = np.asarray(X).reshape(-1, window_size, 3)
y = np.asarray(y)

ValueError: cannot reshape array of size 39983125 into shape (125,3)

In [91]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify = y)

In [92]:
# Define model
model = Sequential()
model.add(Conv1D(64, 3, activation='relu', input_shape=(125, 3)))
model.add(Conv1D(64, 3, activation='relu'))
model.add(MaxPooling1D(3))
model.add(Conv1D(128, 3, activation='relu'))
model.add(Conv1D(128, 3, activation='relu'))
model.add(MaxPooling1D(3))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(11, activation='softmax')) # Change this to the number of classes you have

# Compile model
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [93]:
history = model.fit(X_train, y_train, epochs = 5, validation_split=0.2, verbose=1)

Epoch 1/5
4696/4696 [==============================] - 35s 8ms/step - loss: 0.2491 - accuracy: 0.9251 - val_loss: 0.1270 - val_accuracy: 0.9612
Epoch 2/5
4696/4696 [==============================] - 35s 8ms/step - loss: 0.1294 - accuracy: 0.9642 - val_loss: 0.0772 - val_accuracy: 0.9779
Epoch 3/5
4696/4696 [==============================] - 38s 8ms/step - loss: 0.0969 - accuracy: 0.9747 - val_loss: 0.0795 - val_accuracy: 0.9784
Epoch 4/5
4696/4696 [==============================] - 36s 8ms/step - loss: 0.0837 - accuracy: 0.9792 - val_loss: 0.0565 - val_accuracy: 0.9870
Epoch 5/5
4696/4696 [==============================] - 38s 8ms/step - loss: 0.0704 - accuracy: 0.9828 - val_loss: 0.0528 - val_accuracy: 0.9868


In [94]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test loss: {loss}, Test accuracy: {accuracy}')

1468/1468 [==============================] - 3s 2ms/step - loss: 0.0570 - accuracy: 0.9860
Test loss: 0.057011134922504425, Test accuracy: 0.9859653115272522


In [95]:
# Convert the model.
import tensorflow as tf
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model.
with open('./ten_sec.tflite', 'wb') as f:
  f.write(tflite_model)

2023-11-20 17:38:56.959973: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,12,128]
	 [[{{node inputs}}]]
2023-11-20 17:38:56.965548: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,100]
	 [[{{node inputs}}]]
2023-11-20 17:38:57.060280: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,12,128]
	 [[{{node inputs}}]]
2023-11-20

INFO:tensorflow:Assets written to: /var/folders/sc/rcjbplcx7kq_sgflgb0zyddm0000gn/T/tmpl7gis1p4/assets


INFO:tensorflow:Assets written to: /var/folders/sc/rcjbplcx7kq_sgflgb0zyddm0000gn/T/tmpl7gis1p4/assets
2023-11-20 17:38:57.894880: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2023-11-20 17:38:57.894891: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2023-11-20 17:38:57.895064: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /var/folders/sc/rcjbplcx7kq_sgflgb0zyddm0000gn/T/tmpl7gis1p4
2023-11-20 17:38:57.896449: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2023-11-20 17:38:57.896458: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /var/folders/sc/rcjbplcx7kq_sgflgb0zyddm0000gn/T/tmpl7gis1p4
2023-11-20 17:38:57.901264: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2023-11-20 17:38:57.937348: I tensorflow/cc/saved_model/loader.cc:215] Running initialization

In [ ]:
interpreter = tf.lite.Interpreter(model_path="../models/cnn_model_task1_50_3.tflite")
interpreter.allocate_tensors()

# Get input and output tensors
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Test the model on one instance
interpreter.set_tensor(input_details[0]['index'], X_test[0:1])
interpreter.invoke()
output_data = interpreter.get_tensor(output_details[0]['index'])
print(output_data)

[[9.8876357e-01 2.4413854e-04 1.8548071e-13 1.1150746e-20 1.4760856e-06
  1.1570693e-11 3.1267182e-06 1.0984417e-02 3.6333811e-12 3.1542600e-06
  6.7151853e-09]]


In [ ]:
correct_predictions = 0
total_predictions = 0

for i in range(len(X_test)):
    interpreter.set_tensor(input_details[0]['index'], X_test[i:i+1])
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])
    total_predictions += 1
    if np.argmax(output_data) == np.argmax(y_test[i]):
        correct_predictions += 1

accuracy = correct_predictions / total_predictions
print('Test accuracy:', accuracy)

Test accuracy: 0.08710033076074973


## Task 2

In [41]:
df_2 = har_df.drop(["Unnamed: 0.1", "Unnamed: 0", "timestamp", "gyro_x", "gyro_y", "gyro_z"], axis=1)

In [50]:
df_2 = df_2[(df_2.Subtype != "laughing") & (df_2.Subtype != "singing") & (df_2.Subtype != "talking") & (df_2.Subtype != "eating")]

In [52]:
df2 = df_2[(df_2.Activity != "shuffleWalking") & (df_2.Activity != "normalWalking") & (df_2.Activity != "ascending") & 
(df_2.Activity != "descending") & (df_2.Activity != "running") & (df_2.Activity != "miscMovement")]

In [55]:
df2.Main_Activity.unique()

array(['lyingBack hyperventilating', 'lyingStomach breathingNormal',
       'lyingLeft hyperventilating', 'lyingRight hyperventilating',
       'lyingBack coughing', 'lyingRight breathingNormal',
       'standing breathingNormal', 'lyingStomach coughing',
       'sitting coughing', 'sitting hyperventilating',
       'sitting breathingNormal', 'standing hyperventilating',
       'standing coughing', 'lyingRight coughing',
       'lyingBack breathingNormal', 'lyingLeft coughing',
       'lyingStomach hyperventilating', 'lyingLeft breathingNormal'],
      dtype=object)

In [56]:
class CustomLabelEncoder_2:
    def __init__(self):
        self.class_mapping = {
    'sitting breathingNormal': 0,
    'standing breathingNormal': 0,
    'lyingLeft breathingNormal': 1,
    'lyingRight breathingNormal': 2,
    'lyingBack breathingNormal': 3,
    'lyingStomach breathingNormal': 4,
    'sitting coughing': 5,
    'standing coughing': 5,
    'lyingLeft coughing': 6,
    'lyingRight coughing': 7,
    'lyingBack coughing': 8,
    'lyingStomach coughing': 9,
    'sitting hyperventilating': 10,
    'standing hyperventilating': 10,
    'lyingLeft hyperventilating': 11,
    'lyingRight hyperventilating': 12,
    'lyingBack hyperventilating': 13,
    'lyingStomach hyperventilating': 14
}

    def fit_transform(self, y):
        return [self.class_mapping[cls] for cls in y]

    def inverse_transform(self, y):
        reverse_mapping = {v: k for k, v in self.class_mapping.items()}
        return [reverse_mapping[val] for val in y]

In [57]:
cle2 = CustomLabelEncoder_2()
df2.Main_Activity = cle2.fit_transform(df2.Main_Activity)

/var/folders/sc/rcjbplcx7kq_sgflgb0zyddm0000gn/T/ipykernel_22901/2864437522.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2.Main_Activity = cle2.fit_transform(df2.Main_Activity)


In [58]:
df2

,accel_x,accel_y,accel_z,Activity,Subtype,Main_Activity
4506,-0.485107,-0.346985,0.909119,lyingBack,hyperventilating,13
4507,-0.484131,-0.342590,0.904968,lyingBack,hyperventilating,13
4508,-0.482422,-0.339905,0.899597,lyingBack,hyperventilating,13
4509,-0.487305,-0.333557,0.884949,lyingBack,hyperventilating,13
4510,-0.487793,-0.346008,0.905212,lyingBack,hyperventilating,13
...,...,...,...,...,...,...
2346321,0.180420,-0.127991,-0.906311,lyingStomach,breathingNormal,4
2346322,0.177490,-0.130188,-0.914368,lyingStomach,breathingNormal,4
2346323,0.182617,-0.129944,-0.911438,lyingStomach,breathingNormal,4
2346324,0.182861,-0.131165,-0.903625,lyingStomach,breathingNormal,4


In [69]:
window_size = 100

stride = 15  # Adjust this as needed

# Create sliding windows and labels
X, y = [], []

for i in range(0, len(df2) - window_size, stride):
    window_data = df2.iloc[i:i + window_size]
    X.append(window_data.drop(['Activity', 'Subtype', 'Main_Activity'], axis=1).values)
    y.append(stats.mode(np.array(window_data['Main_Activity'])).mode)
    # y.append(window_data['Activity'].values[-1])
X = np.asarray(X).reshape(-1, window_size, 3)
y = np.asarray(y)

In [70]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify = y)

In [73]:
m = Sequential()
m.add(Conv1D(64, 3, activation='relu', input_shape=(100, 3)))
m.add(Conv1D(64, 3, activation='relu'))
m.add(MaxPooling1D(3))
m.add(Conv1D(128, 3, activation='relu'))
m.add(Conv1D(128, 3, activation='relu'))
m.add(MaxPooling1D(3))
m.add(Dropout(0.5))
m.add(Flatten())
m.add(Dense(100, activation='relu'))
m.add(Dropout(0.5))
m.add(Dense(15, activation='softmax')) # Change this to the number of classes you have

# Compile model
m.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [75]:
history = m.fit(X_train, y_train, epochs = 20, validation_split=0.2, verbose=1)

Epoch 1/20
1278/1278 [==============================] - 8s 7ms/step - loss: 0.2775 - accuracy: 0.8904 - val_loss: 0.2304 - val_accuracy: 0.9082
Epoch 2/20
1278/1278 [==============================] - 9s 7ms/step - loss: 0.2598 - accuracy: 0.8987 - val_loss: 0.2428 - val_accuracy: 0.9048
Epoch 3/20
1278/1278 [==============================] - 8s 6ms/step - loss: 0.2548 - accuracy: 0.9003 - val_loss: 0.2512 - val_accuracy: 0.9076
Epoch 4/20
1278/1278 [==============================] - 8s 6ms/step - loss: 0.2482 - accuracy: 0.9017 - val_loss: 0.2444 - val_accuracy: 0.9080
Epoch 5/20
1278/1278 [==============================] - 8s 6ms/step - loss: 0.2424 - accuracy: 0.9055 - val_loss: 0.2083 - val_accuracy: 0.9179
Epoch 6/20
1278/1278 [==============================] - 8s 6ms/step - loss: 0.2400 - accuracy: 0.9051 - val_loss: 0.1898 - val_accuracy: 0.9310
Epoch 7/20
1278/1278 [==============================] - 8s 6ms/step - loss: 0.2259 - accuracy: 0.9115 - val_loss: 0.2033 - val_accuracy:

In [76]:
loss, accuracy = m.evaluate(X_test, y_test)
print(f'Test loss: {loss}, Test accuracy: {accuracy}')

400/400 [==============================] - 1s 2ms/step - loss: 0.1528 - accuracy: 0.9460
Test loss: 0.15277980268001556, Test accuracy: 0.9459755420684814


In [78]:
# Convert the model.
import tensorflow as tf
converter = tf.lite.TFLiteConverter.from_keras_model(m)
tflite_model = converter.convert()

# Save the model.
with open('../ML Model/task2.tflite', 'wb') as f:
  f.write(tflite_model)

2023-11-20 17:04:56.344365: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,9,128]
	 [[{{node inputs}}]]
2023-11-20 17:04:56.349845: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,100]
	 [[{{node inputs}}]]
2023-11-20 17:04:56.443194: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,9,128]
	 [[{{node inputs}}]]
2023-11-20 1

INFO:tensorflow:Assets written to: /var/folders/sc/rcjbplcx7kq_sgflgb0zyddm0000gn/T/tmpo5d120qm/assets


INFO:tensorflow:Assets written to: /var/folders/sc/rcjbplcx7kq_sgflgb0zyddm0000gn/T/tmpo5d120qm/assets
2023-11-20 17:04:57.253721: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2023-11-20 17:04:57.253735: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2023-11-20 17:04:57.253875: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /var/folders/sc/rcjbplcx7kq_sgflgb0zyddm0000gn/T/tmpo5d120qm
2023-11-20 17:04:57.255201: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2023-11-20 17:04:57.255208: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /var/folders/sc/rcjbplcx7kq_sgflgb0zyddm0000gn/T/tmpo5d120qm
2023-11-20 17:04:57.259476: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2023-11-20 17:04:57.293401: I tensorflow/cc/saved_model/loader.cc:215] Running initialization